In [1]:
import requests
import pandas as pd
import concurrent.futures

In [2]:
league_id = '366970'
general_endpoint = "https://fantasy.premierleague.com/api/bootstrap-static/"
league_endpoint = f'https://fantasy.premierleague.com/api/leagues-classic/{league_id}/standings'

In [3]:
def fetch_data(general_endpoint, league_endpoint):
    general_response = requests.get(general_endpoint).json()
    league_response = requests.get(league_endpoint).json()
    real_player_data = pd.DataFrame.from_records(general_response['elements'], columns=['id', 'web_name', 'selected_by_percent', 'total_points', 'bonus', 'form', 'value_season', 'value_form' ]).astype({"form": float, "value_season": float, "value_form": float}).set_index('id')
    gameweek_data = pd.DataFrame.from_records(general_response['events'], columns=['id', "deadline_time",'is_current', 'most_captained', 'most_vice_captained', 'top_element']).fillna(0).astype({"most_captained": int, "most_vice_captained": int, "top_element": int}).set_index('id')
    current_standings = pd.DataFrame.from_records(league_response['standings']['results'], columns=['id', 'entry', 'player_name', 'rank', 'last_rank', 'total']).astype({"rank": int, "last_rank": int, "total": int}).set_index('entry')
    
    
    current_week = gameweek_data.loc[gameweek_data['is_current'] == True].index.values[0]
    counter = 0
    for i in current_standings.index:
        if counter == 0:
            endpt = f'https://fantasy.premierleague.com/api/entry/{i}/event/{current_week}/picks/'
            re = requests.get(endpt).json()
            manager_teams = pd.DataFrame.from_records(re['picks'], columns=['element'])
            manager_teams["entry"] = i

        else: 
            endpt = f'https://fantasy.premierleague.com/api/entry/{i}/event/{current_week}/picks/'
            re = requests.get(endpt).json()
            append_df = pd.DataFrame.from_records(re['picks'], columns=['element'])
            append_df["entry"] = i
            manager_teams = pd.concat([manager_teams, append_df])
        counter += 1

    current_gameweek_teams= pd.merge(manager_teams, real_player_data, left_on='element', right_on='id')


    return gameweek_data, current_standings, current_gameweek_teams

In [14]:
def fetch_data(general_endpoint, league_endpoint):
    with requests.Session() as session:
        general_response = session.get(general_endpoint).json()
        league_response = session.get(league_endpoint).json()

    real_player_data = pd.DataFrame.from_records(general_response['elements'], columns=['id', 'web_name', 'selected_by_percent', 'total_points', 'bonus', 'form', 'value_season', 'value_form' ]).astype({"form": float, "value_season": float, "value_form": float}).set_index('id')
    gameweek_data = pd.DataFrame.from_records(general_response['events'], columns=['id', "deadline_time",'is_current', 'most_captained', 'most_vice_captained', 'top_element']).fillna(0).astype({"most_captained": int, "most_vice_captained": int, "top_element": int}).set_index('id')
    current_standings = pd.DataFrame.from_records(league_response['standings']['results'], columns=['id', 'entry', 'player_name', 'rank', 'last_rank', 'total']).astype({"rank": int, "last_rank": int, "total": int}).set_index('entry')
    current_week = gameweek_data.index[gameweek_data['is_current']].item()
    manager_teams = pd.concat([pd.DataFrame.from_records(session.get(f'https://fantasy.premierleague.com/api/entry/{i}/event/{current_week}/picks/').json()['picks'], ).assign(entry=i) for i in current_standings.index])
    current_gameweek_teams = pd.merge(manager_teams, real_player_data, left_on='element', right_index=True)

    return gameweek_data, current_standings, current_gameweek_teams

In [15]:
gameweek_data, current_standings, current_gameweek_teams = fetch_data(general_endpoint, league_endpoint)

In [16]:
current_gameweek_teams.head()

,element,position,multiplier,is_captain,is_vice_captain,entry,web_name,selected_by_percent,total_points,bonus,form,value_season,value_form
0,81,1,1,False,False,2208836,Raya,16.2,144,20,4.0,29.4,0.8
11,81,12,0,False,False,552345,Raya,16.2,144,20,4.0,29.4,0.8
0,81,1,1,False,False,578302,Raya,16.2,144,20,4.0,29.4,0.8
0,81,1,1,False,False,560769,Raya,16.2,144,20,4.0,29.4,0.8
0,81,1,1,False,False,1930701,Raya,16.2,144,20,4.0,29.4,0.8


In [39]:
x = current_gameweek_teams.iloc[:, [5,7]].astype({"selected_by_percent": float})

a = pd.DataFrame(x.groupby('entry')["selected_by_percent"].mean().sort_values(ascending=False))

a.merge(current_standings, on='entry').iloc[:, [0,1,2]]

,selected_by_percent,id,player_name
entry,,,
578302,28.120000,42389877,Ryan Shacks
534613,27.280000,11045904,Connor McDonald
1789073,26.920000,11595521,Devon Jansen
2208836,26.760000,54837678,Divyam Dixit
560769,26.580000,11044918,Alex Wietzorrek
1742534,26.520000,10303098,Niklas Wietzorrek
1002132,26.506667,11163290,Devon Hodgson
2123591,24.793333,42326189,Peter Wertz
1947717,24.413333,11600435,Cole Floyd


In [12]:
gameweek_data.head()

,deadline_time,is_current,most_captained,most_vice_captained,top_element
id,,,,,
1,2022-08-05T17:30:00Z,False,283,318,366
2,2022-08-13T10:00:00Z,False,318,283,28
3,2022-08-20T10:00:00Z,False,28,318,7
4,2022-08-27T10:00:00Z,False,283,28,279
5,2022-08-30T17:00:00Z,False,318,28,318


In [13]:
current_standings.head()

,id,player_name,rank,last_rank,total
entry,,,,,
2208836,54837678,Divyam Dixit,1,2,2117
552345,11604984,Marco Gouveia,2,1,2093
534613,11045904,Connor McDonald,3,3,2031
578302,42389877,Ryan Shacks,4,4,2026
560769,11044918,Alex Wietzorrek,5,5,2019


In [40]:
21.2+12.7+67.2+15+37.6+43.3+33.1+14.4+81+23+4.8+7.9+17.7+14.3+16


409.2